# PyTorch之牛刀小试
- (1)一个n张量，类似于Numpy，但可以在GPU上运行
- (2)搭建和训练神经网络时的自动微分/求导机制

## 张量
Numpy是一个很棒的框架，但它不能利用GPU来加速其数值计算。 对于现代深度神经网络，GPU通常提供50倍或更高的加速，所以，numpy不能满足当代深度学习的需求。

在这里先介绍一下PyTorch的概念:  

张量(Tensor)：PyTorch的tensor在概念上与numpy的array相同，tensor是一个n维数组，PyTorch提供了许多函数用于操作这些张量。任何希望使用Numpy执行的计算也可以使用PyTorch的tensor来完成，可以认为它们是科学计算的通用工具。  

但是与Numpy不同的是，PyTorch可以利用GPU加速其数值计算，要在GPU上运行Tensor，在构造张量使用device参数把tensor建立在GPU上。在这里，使用tensor将随机数据上训练一个两层的网络，这里先手动实现前向传播和反向传播:

In [19]:
import torch

# fixme: 初始配置
dtype = torch.float
# device = torch.device("cpu")＃取消注释以在CPU上运
device = torch.device('cuda:0')
N, D_in, H, D_out = 64, 1000, 100, 10 # N为批量大小，D_in是输入维度，H是隐藏层维度，D_out是输出层维度 (这里构建的两层网络)
learning_rate = 1e-6 # 学习速率
epochs = 100 # 代数

# fixme: 创建随机输入和输出
input = torch.randn(N, D_in, device=device, dtype=dtype)
label = torch.randn(N, D_out, device=device,dtype=dtype)

# fixme: 随机初始化权重
w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device,dtype=dtype)

# fixme: 训练
for epoch in range(epochs):
    # 前向传播部分
    h = input.mm(w1) # 输入层到隐藏层
    h_relu = h.clamp(min=0) # 类似relu函数
    pred = h_relu.mm(w2) # 隐藏层到输出层
    
    # 计算损失并且打印损失
    loss = (pred - label).pow(2).sum().item() # 差的平方求和
    # print('当前代数:',epoch,'，当前loss为:',loss)
    print('当前代数:{}，当前loss为:{}'.format(epoch,loss))
    
    # 反向传播，计算梯度(推导公式模式识别里面讲过)
    grad_pre = 2.0 * (pred - label)
    grad_w2 = h_relu.t().mm(grad_pre)
    
    grad_h_relu = grad_pre.mm(w2.t())
    grad_h = grad_h_relu.clone()
    grad_h[h < 0] = 0
    
    grad_w1 = input.t().mm(grad_h)
    
    # 使用学习速率更新权重
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

当前代数:0，当前loss为:31054268.0
当前代数:1，当前loss为:32792080.0
当前代数:2，当前loss为:40690232.0
当前代数:3，当前loss为:47072136.0
当前代数:4，当前loss为:43159876.0
当前代数:5，当前loss为:27847366.0
当前代数:6，当前loss为:12906761.0
当前代数:7，当前loss为:5127627.0
当前代数:8，当前loss为:2330071.5
当前代数:9，当前loss为:1383562.0
当前代数:10，当前loss为:1009468.875
当前代数:11，当前loss为:813189.875
当前代数:12，当前loss为:682103.75
当前代数:13，当前loss为:582411.75
当前代数:14，当前loss为:502093.40625
当前代数:15，当前loss为:435708.875
当前代数:16，当前loss为:380028.4375
当前代数:17，当前loss为:332927.09375
当前代数:18，当前loss为:292841.8125
当前代数:19，当前loss为:258505.203125
当前代数:20，当前loss为:228938.359375
当前代数:21，当前loss为:203353.5
当前代数:22，当前loss为:181149.84375
当前代数:23，当前loss为:161809.625
当前代数:24，当前loss为:144882.734375
当前代数:25，当前loss为:130003.984375
当前代数:26，当前loss为:116890.46875
当前代数:27，当前loss为:105295.953125
当前代数:28，当前loss为:95030.5625
当前代数:29，当前loss为:85919.65625
当前代数:30，当前loss为:77802.578125
当前代数:31，当前loss为:70554.015625
当前代数:32，当前loss为:64070.0390625
当前代数:33，当前loss为:58257.4375
当前代数:34，当前loss为:53037.4765625
当前代数:35，当前loss为:48341.96875
当前代数:36

## 自动求导
### 张量和自动求导
上面的例子中，我们手动实现了前向传播和后向传播，手动实现对于小型的网络来说还是可行的，但是如果对于实现大型复杂的网络来说那将变得十分复杂，但是可以使用自动微分来自动计算神经网络中的后向传播。  
PyTorch中的autograd包提供了这个功能，**当使用autograd时，网络前向传播将定义一个计算图，图中的节点是tensor，边是函数，这些函数是输出tensor到输入tensor的映射**。这张计算图使得在网络中反向传播时梯度的计算十分简单。  

上面的描述可能听起来十分复杂，但是在实践中非常简单，我们如果想要计算某些tensor的梯度，我们只需要在建立这个tensor时加入一句： **requires_grad=True**，这个tensor上的任何PyTorch的操作都将构建一个计算图，从而允许我们稍后在图中执行反向传播。如果这个tensor x的requires_grad=True，那么反向传播之后，x.grad将会是另一个张量，其为x关于某个标量值的梯度。  
有时可能希望防止PyTorch在requires_grad=True的张量执行某些操作时，构建计算图；例如，在训练神经网络时，我们通常不希望通过权重更新步骤进行反向传播，这种情况下，我们可以使用torch.no_grad()上下文管理器来防止构建计算图。

**默认情况下,所有的tensor的requires_grad均为True**

下面是一个例子:

In [24]:
import torch

# fixme: 配置
dtype = torch.float
device = torch.device('cuda:0')
N, D_in, H, D_out = 64, 1000, 100, 10 # N为批量大小，D_in是输入维度，H是隐藏层维度，D_out是输出层维度 (这里构建的两层网络)
learning_rate = 1e-6 # 学习速率
epochs = 100 # 代数
# fixme: 创建随机输入和输出
input = torch.randn(N, D_in, device=device, dtype=dtype, requires_grad=True)
label = torch.randn(N, D_out, device=device,dtype=dtype, requires_grad=True)

# fixme: 随机初始化权重
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device,dtype=dtype, requires_grad=True)

# fixme: 训练
for epoch in range(epochs):
    # 前向传播部分
    h = input.mm(w1) # 输入层到隐藏层
    h_relu = h.clamp(min=0) # 类似relu函数
    pred = h_relu.mm(w2) # 隐藏层到输出层
    
    # 计算损失并且打印损失
    loss = (pred - label).pow(2).sum() # 差的平方求和
    # print('当前代数:',epoch,'，当前loss为:',loss)
    print('当前代数:{}，当前loss为:{}'.format(epoch,loss.item()))
    
    # 反向传播
    loss.backward()
    
    # 更新梯度
    ## 不更新计算图
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        
        # 手动将梯度设置为0
        w1.grad.zero_()
        w2.grad.zero_()

当前代数:0，当前loss为:27810032.0
当前代数:1，当前loss为:24036258.0
当前代数:2，当前loss为:22474504.0
当前代数:3，当前loss为:20527414.0
当前代数:4，当前loss为:17089804.0
当前代数:5，当前loss为:12828048.0
当前代数:6，当前loss为:8727194.0
当前代数:7，当前loss为:5624930.0
当前代数:8，当前loss为:3571583.75
当前代数:9，当前loss为:2328650.5
当前代数:10，当前loss为:1592575.375
当前代数:11，当前loss为:1153773.75
当前代数:12，当前loss为:881616.625
当前代数:13，当前loss为:703650.875
当前代数:14，当前loss为:580100.5625
当前代数:15，当前loss为:489431.375
当前代数:16，当前loss为:419695.375
当前代数:17，当前loss为:364094.375
当前代数:18，当前loss为:318578.28125
当前代数:19，当前loss为:280553.78125
当前代数:20，当前loss为:248364.734375
当前代数:21，当前loss为:220813.15625
当前代数:22，当前loss为:197072.859375
当前代数:23，当前loss为:176429.609375
当前代数:24，当前loss为:158397.40625
当前代数:25，当前loss为:142566.15625
当前代数:26，当前loss为:128629.28125
当前代数:27，当前loss为:116309.9765625
当前代数:28，当前loss为:105392.078125
当前代数:29，当前loss为:95699.6875
当前代数:30，当前loss为:87067.65625
当前代数:31，当前loss为:79355.140625
当前代数:32，当前loss为:72449.046875
当前代数:33，当前loss为:66244.546875
当前代数:34，当前loss为:60659.79296875
当前代数:35，当前loss为:55621.03515

### 定义新的自动求导函数
在底层，每一个原始的自动求导运算实际上是两个在Tensor上运行的函数。其中， forward 函数计算从输入Tensors获得的输出Tensors。而 backward 函数接收输出Tensors对于某个标量值的梯度，并且计算输入Tensors相对于该相同标量值的梯度。  

在PyTorch中，我们可以很容易地通过定义 torch.autograd.Function 的子类并实现 forward 和backward函数，来定义自己的自动求导运算。之后我们就可以使用这个新的自动梯度运算符了。然后，我们可以通过构造一个实例并像调用函数一样，传入包含输入数据的tensor调用它，这样来使用新的自动求导运算。  

下面是一个例子:

In [25]:
import torch

class MyReLU(torch.autograd.Function):
    """
        通过建立torch.autograd的子类来实现我们自定义的autograd函数，并且完成张量的正向和反向传播。
    """
    @staticmethod
    def forward(ctx, x):
        """
        在正向传播中，我们接收到一个上下文对象和一个包含输入的张量；
        我们必须返回一个包含输出的张量，并且我们可以使用上下文对象来缓存对象，
        以便在反向传播中使用。
        """
        ctx.save_for_backward(x)
        return x.clamp(min=0)
    
    @staticmethod
    def backward(ctx, grad_output):
        """
        在反向传播中，我们接收到上下文对象和一个张量，
        其包含了相对于正向传播过程中产生的输出的损失的梯度。
        我们可以从上下文对象中检索缓存的数据，
        并且必须计算并返回与正向传播的输入相关的损失的梯度。
        """
        x, = ctx.saved_tensors
        grad_x = grad_output.clone()
        grad_x[x < 0] = 0
        return grad_x

### TensorFlow：静态图
PyTorch自动求导看起来非常像TensorFlow，这两个框架中，我们都定义了计算图，使用自动微分来计算梯度，但是两者之间最大的不同是TensorFlow的计算图是静态的，而PyTorch使用的是动态的计算图。    
**在TensorFlow中，我们定义计算图一次，然后后续就会重复执行这个相同的图，后面的话可能只是会提供不同的输入数据，而在PyTorch中，每一个前向通道(forward)定义一个新的计算图。**  

静态图的好处在于你可以预先对图进行优化。例如：一个框架可能要融合一些图的运算来提升效率，或者产生一个策略来将图分布到多个GPU或者机器上，如果重复使用相同的图，那么再重复运行一个图时，前期潜在的代价高昂的预先优化的消耗就会被分摊开。  

静态图和动态图的一个区别是**控制流**。对于一些模型，我们希望对每个数据点执行不同的计算。例如：一个递归神经网络可能对每个数据点执行不同的时间步数，这个展开（unrolling)可以作为一个循环来实现。  
对于一个静态图，循环结构要作为图的一部分，因此TensorFlow提供了运算符来把循环嵌入到图当中。对于动态图来说，情况更加简单，既然我们为每个例子即时创建计算图，我们可以使用普通的命令式控制流来为每个输入执行不同的计算。  

## nn模块
### torch.nn
计算图和autograd是十分强大的工具，可以定义复杂的操作并自动求导，然后对于大规模的网络来说，autograd又太过于底层了，在构建神经网络时，我们经常考虑将计算安排成层，其中一些具有可学习性的参数，让它们在学习过程中进行优化。  

在TensorFlow中，有类似Keras，TensorFlow-Slim和TFLearn这种封装了底层计算图的高度抽象的接口，这使得构建网络十分方便。  

在PyTorch中，包nn完成了和TFLearn等类似的功能，nn包中定义一组大致等价于层的块。一个模块接受输入的tensor，计算输出的tensor，而且还保存了一些内部状态，比如需要学习的tensor的参数等。nn包中也定义了一组损失函数，用来训练神经网络。

下面给出一个用nn包实现两层网络的例子:

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# fixme: 参数配置
dtype = torch.float
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
N, D_in, H, D_out = 64, 1000, 100, 10 # N为批量大小，D_in是输入维度，H是隐藏层维度，D_out是输出层维度
learning_rate = 1e-4
epoches = 100

# fixeme: 创建输入和输出随机张量
input = torch.randn(N, D_in)
label = torch.randn(N, D_out)

# fixme: 使用nn包将我们的模型定义为一系列的层
## nn.Sequential是包含其他模块的模块，并按顺序应用这些模块来产生其输出。
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)

# nn包还包含常用的损失函数的定义
# 在这种情况下，我们将使用平均平方误差(MSE)作为我们的损失函数
# 设置reduction='sum'，表示我们计算的是平方误差的“和”，而不是平均值;
# 但是在实践中，通过设置reduction='elementwise_mean'来使用均方误差作为损失更为常见。
loss_fn = torch.nn.MSELoss(reduction='sum')

for epoch in range(epoches):
    # 前向传播
    pred = model(input)
    
    # 计算loss
    loss = loss_fn(pred, label)
    print('当前代数:{}，当前loss为:{}'.format(epoch,loss.item()))
    
    # 反向传播梯度清零
    model.zero_grad()
    
    # 反向传播
    loss.backward()
    
    
    # 更新梯度
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

当前代数:0，当前loss为:694.401123046875
当前代数:1，当前loss为:640.5441284179688
当前代数:2，当前loss为:594.4757690429688
当前代数:3，当前loss为:554.1473388671875
当前代数:4，当前loss为:518.3447875976562
当前代数:5，当前loss为:486.1473388671875
当前代数:6，当前loss为:457.0733337402344
当前代数:7，当前loss为:430.4808654785156
当前代数:8，当前loss为:406.1338195800781
当前代数:9，当前loss为:383.46478271484375
当前代数:10，当前loss为:362.3720703125
当前代数:11，当前loss为:342.73736572265625
当前代数:12，当前loss为:324.1927185058594
当前代数:13，当前loss为:306.5917053222656
当前代数:14，当前loss为:289.88323974609375
当前代数:15，当前loss为:274.0377197265625
当前代数:16，当前loss为:259.0152587890625
当前代数:17，当前loss为:244.71998596191406
当前代数:18，当前loss为:231.1725311279297
当前代数:19，当前loss为:218.3126983642578
当前代数:20，当前loss为:206.06178283691406
当前代数:21，当前loss为:194.44424438476562
当前代数:22，当前loss为:183.42593383789062
当前代数:23，当前loss为:172.9327392578125
当前代数:24，当前loss为:162.96517944335938
当前代数:25，当前loss为:153.50523376464844
当前代数:26，当前loss为:144.53953552246094
当前代数:27，当前loss为:136.04568481445312
当前代数:28，当前loss为:128.0102996826172
当前代数:29，当前loss为:1

### torch.optim
到目前为止，我们已经通过手动改变包含可学习参数的张量来更新模型的权重。对于随机梯度下降(SGD/stochastic gradient descent)等简单的优化算法来说，这不是一个很大的负担，但在实践中，我们经常使用AdaGrad、RMSProp、Adam等更复杂的优化器来训练神经网络。

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# fixme: 参数配置
dtype = torch.float
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
N, D_in, H, D_out = 64, 1000, 100, 10 # N为批量大小，D_in是输入维度，H是隐藏层维度，D_out是输出层维度
learning_rate = 1e-4
epochs = 100

# fixeme: 创建输入和输出随机张量
input = torch.randn(N, D_in)
label = torch.randn(N, D_out)

# fixme: 使用nn包将我们的模型定义为一系列的层
## nn.Sequential是包含其他模块的模块，并按顺序应用这些模块来产生其输出。
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)

# fixme: nn包还包含常用的损失函数的定义
# 在这种情况下，我们将使用平均平方误差(MSE)作为我们的损失函数
# 设置reduction='sum'，表示我们计算的是平方误差的“和”，而不是平均值;
# 但是在实践中，通过设置reduction='elementwise_mean'来使用均方误差作为损失更为常见。
loss_fn = torch.nn.MSELoss(reduction='sum')

# fixme: 使用torch.optim定义参数优化器
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate) # 这里用的是Adam

for epoch in range(epochs):
    # 前向传播
    pred = model(input)
    
    # 计算loss
    loss = loss_fn(pred, label)
    print('当前代数:{}，当前loss为:{}'.format(epoch,loss.item()))
    
    # 反向传播优化器中的模型参数梯度清零
    optimizer.zero_grad()
    
    # 反向传播
    loss.backward()
    
    # 调用Optimizer的step函数使它所有的参数更新
    optimizer.step()

当前代数:0，当前loss为:671.5050048828125
当前代数:1，当前loss为:654.7916870117188
当前代数:2，当前loss为:638.5562133789062
当前代数:3，当前loss为:622.7379760742188
当前代数:4，当前loss为:607.2938842773438
当前代数:5，当前loss为:592.3021850585938
当前代数:6，当前loss为:577.6829223632812
当前代数:7，当前loss为:563.436279296875
当前代数:8，当前loss为:549.675048828125
当前代数:9，当前loss为:536.265869140625
当前代数:10，当前loss为:523.209228515625
当前代数:11，当前loss为:510.4918212890625
当前代数:12，当前loss为:498.1060791015625
当前代数:13，当前loss为:486.1214599609375
当前代数:14，当前loss为:474.44940185546875
当前代数:15，当前loss为:463.1405029296875
当前代数:16，当前loss为:452.1324157714844
当前代数:17，当前loss为:441.3900146484375
当前代数:18，当前loss为:431.0217590332031
当前代数:19，当前loss为:420.8746337890625
当前代数:20，当前loss为:410.96966552734375
当前代数:21，当前loss为:401.34454345703125
当前代数:22，当前loss为:391.9026794433594
当前代数:23，当前loss为:382.7084655761719
当前代数:24，当前loss为:373.7532043457031
当前代数:25，当前loss为:364.9910888671875
当前代数:26，当前loss为:356.41558837890625
当前代数:27，当前loss为:348.0168762207031
当前代数:28，当前loss为:339.8182067871094
当前代数:29，当前loss为:331.8048

### 自定义nn模块
有时候需要制定比现有模块序列更加复杂的模型，对于这些情况，可以通过继承nn.Module并定义forward函数，这个forward函数可以使用其他模块或者其他的自动求导运算来接收输入tensor，产生输出tensor。  

下面是一个例子:

In [13]:
import torch
import torch.nn as nn

# fixme: 构建网络
class TwoLayerNet(nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        构造函数，在这里需要将网络的各个模块进行实例化, 并把他们作为成员变量
        :params D_in: 输入维度
        :params H: 隐藏层维度
        :params D_out: 输出维度
        """
        super(TwoLayerNet, self).__init__()
        
        self.linear1 = nn.Linear(D_in, H)
        self.linear2 = nn.Linear(H, D_out)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        """
        构造前向传播函数
        """
        x1 = self.linear1(x)
        x_relu = self.relu(x1)
        pred = self.linear2(x_relu)
        
        return pred

# fixme: 参数配置
dtype = torch.float
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
N, D_in, H, D_out = 64, 1000, 100, 10 # N为批量大小，D_in是输入维度，H是隐藏层维度，D_out是输出层维度
learning_rate = 1e-4
epochs = 100

# fixeme: 创建输入和输出随机张量
input = torch.randn(N, D_in)
label = torch.randn(N, D_out)

# fixme: 实例化模型
model = TwoLayerNet(D_in, H, D_out)

# fixme: 损失函数的定义
loss_fn = torch.nn.MSELoss(reduction='sum')

# fixme: 使用torch.optim定义参数优化器
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate) # 这里用的是Adam

# fixme: 训练
for epoch in range(epochs):
    # 前向过程
    pred = model(input)
    
    # 计算loss
    loss = loss_fn(pred, label)
    print('当前代数:{}，当前loss为:{}'.format(epoch,loss.item()))
    
    # 模型参数梯度置零
    optimizer.zero_grad()
    
    # loss反向传播
    loss.backward()
    
    # 更新权重
    optimizer.step()

当前代数:0，当前loss为:661.89208984375
当前代数:1，当前loss为:644.700439453125
当前代数:2，当前loss为:627.9445190429688
当前代数:3，当前loss为:611.715087890625
当前代数:4，当前loss为:595.9672241210938
当前代数:5，当前loss为:580.7451171875
当前代数:6，当前loss为:565.9742431640625
当前代数:7，当前loss为:551.5748291015625
当前代数:8，当前loss为:537.4945678710938
当前代数:9，当前loss为:523.865966796875
当前代数:10，当前loss为:510.6148986816406
当前代数:11，当前loss为:497.7690734863281
当前代数:12，当前loss为:485.2662353515625
当前代数:13，当前loss为:473.086669921875
当前代数:14，当前loss为:461.2438659667969
当前代数:15，当前loss为:449.75933837890625
当前代数:16，当前loss为:438.7503662109375
当前代数:17，当前loss为:428.12249755859375
当前代数:18，当前loss为:417.7576904296875
当前代数:19，当前loss为:407.6864318847656
当前代数:20，当前loss为:397.8720703125
当前代数:21，当前loss为:388.26007080078125
当前代数:22，当前loss为:378.8917236328125
当前代数:23，当前loss为:369.71429443359375
当前代数:24，当前loss为:360.8055419921875
当前代数:25，当前loss为:352.11920166015625
当前代数:26，当前loss为:343.6255798339844
当前代数:27，当前loss为:335.3822937011719
当前代数:28，当前loss为:327.343994140625
当前代数:29，当前loss为:319.523956298828

### 控制流和权重共享
作为动态图(网络结构发生变化并不影响计算图计算梯度)和权重共享的一个例子，我们实现了一个非常奇怪的模型：一个全连接的ReLU网络，在每一次前向传播时，它的隐藏层的层数为随机1到4之间的数，这样可以多次重用相同的权重来计算。  

因为这个模型可以使用普通的Python流控制来实现循环，并且我们可以通过定义转发时多次重用同一个模块来实现最内层的权重共享。

下面是例子的代码:

In [14]:
import torch
import torch.nn as nn
import random

# fixme: 定义网络
class DynamicNet(nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        构造函数，在这里需要将网络的各个模块进行实例化, 并把他们作为成员变量
        :params D_in: 输入维度
        :params H: 隐藏层维度
        :params D_out: 输出维度
        """
        super(DynamicNet, self).__init__()
        
        self.input_layer = nn.Linear(D_in, H)
        self.hidden_layer = nn.Linear(H, H)
        self.output_layer = nn.Linear(H, D_out)
        
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x1 = self.input_layer(x)
        t_relu = self.relu(x1)
        
        # 定义0-3个隐藏层，利用pytorch动态图的特征，这种做法是可行的
        ## 重复调用self.hidden_layer 0-3次，由于pytorch是采用动态图的，因此每一次forward都会创建一个新的动态图，不影响梯度计算
        for _ in range(random.randint(0, 3)):
            t_relu = self.relu(self.hidden_layer(t_relu))
        
        pred = self.output_layer(t_relu)
        
        return pred

# fixme: 参数配置
dtype = torch.float
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
N, D_in, H, D_out = 64, 1000, 100, 10 # N为批量大小，D_in是输入维度，H是隐藏层维度，D_out是输出层维度
learning_rate = 1e-4
epochs = 100

# fixeme: 创建输入和输出随机张量
input = torch.randn(N, D_in)
label = torch.randn(N, D_out)

# fixme: 实例化模型
model = DynamicNet(D_in, H, D_out)

# fixme: 损失函数的定义
loss_fn = torch.nn.MSELoss(reduction='sum')

# fixme: 使用torch.optim定义参数优化器
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate) # 这里用的是Adam

# fixme: 训练
for epoch in range(epochs):
    # 前向过程
    pred = model(input)
    
    # 计算loss
    loss = loss_fn(pred, label)
    print('当前代数:{}，当前loss为:{}'.format(epoch,loss.item()))
    
    # 模型参数梯度置零
    optimizer.zero_grad()
    
    # loss反向传播
    loss.backward()
    
    # 更新权重
    optimizer.step()

当前代数:0，当前loss为:702.3717651367188
当前代数:1，当前loss为:700.1735229492188
当前代数:2，当前loss为:697.996826171875
当前代数:3，当前loss为:701.37646484375
当前代数:4，当前loss为:745.0324096679688
当前代数:5，当前loss为:686.2260131835938
当前代数:6，当前loss为:729.3468017578125
当前代数:7，当前loss为:719.3234252929688
当前代数:8，当前loss为:700.3496704101562
当前代数:9，当前loss为:697.38720703125
当前代数:10，当前loss为:676.840576171875
当前代数:11，当前loss为:674.8969116210938
当前代数:12，当前loss为:667.5950927734375
当前代数:13，当前loss为:657.723388671875
当前代数:14，当前loss为:668.5684814453125
当前代数:15，当前loss为:691.20361328125
当前代数:16，当前loss为:629.1569213867188
当前代数:17，当前loss为:662.6659545898438
当前代数:18，当前loss为:611.5732421875
当前代数:19，当前loss为:602.4924926757812
当前代数:20，当前loss为:698.65625
当前代数:21，当前loss为:583.9902954101562
当前代数:22，当前loss为:655.1369018554688
当前代数:23，当前loss为:566.2965087890625
当前代数:24，当前loss为:557.3516845703125
当前代数:25，当前loss为:547.8927001953125
当前代数:26，当前loss为:650.374755859375
当前代数:27，当前loss为:648.9633178710938
当前代数:28，当前loss为:697.7493286132812
当前代数:29，当前loss为:515.4136962890625
当前代数:30，当前l